<a href="https://colab.research.google.com/github/YasudaKaito/introduction_llm/blob/main/ch3_llm_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[ja,sentencepiece,torch] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 24.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-

In [3]:
from transformers import pipeline

In [ ]:
# GPTでテキスト生成
# 後続テキストを予測するpipelineを作成
generator = pipeline(
    "text-generation",
    model="abeja/gpt2-large-japanese",
)
outputs = generator("日本で一番高い山は")
print(outputs)

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '日本で一番高い山は富士山、2番目に高い山は富士山本宮浅間大社(ふもとのみやせんげんはくさんせんげんしゃ)です。 この2つの山は、日本五大桜山とも呼ばれ、国の名勝にも'}]


In [ ]:
import pandas as pd

# BERTで穴埋めを解く
fill_mask = pipeline(
    "fill-mask",
    model="cl-tohoku/bert-base-japanese-v3",
)

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [ ]:
masked_text = "日本の首都は [MASK] である"
outputs = fill_mask(masked_text)
display(pd.DataFrame(outputs[:10]))

,score,token,token_str,sequence
0,0.884170,12569,東京,日本 の 首都 は 東京 で ある
1,0.024820,12759,大阪,日本 の 首都 は 大阪 で ある
2,0.020864,13017,京都,日本 の 首都 は 京都 で ある
3,0.004853,14499,奈良,日本 の 首都 は 奈良 で ある
4,0.004590,14424,仙台,日本 の 首都 は 仙台 で ある


In [ ]:
masked_text = "今日の映画は刺激的で面白かった。この映画は [MASK]。"
outputs = fill_mask(masked_text)
display(pd.DataFrame(outputs[:5]))

,score,token,token_str,sequence
0,0.683933,23845,素晴らしい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 素晴らしい 。
1,0.101234,24683,面白い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 面白い 。
2,0.048003,26840,楽しい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 楽しい 。
3,0.034067,30790,すごい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は すごい 。
4,0.025223,17707,美しい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 美しい 。


In [5]:
# T5でスパンの穴埋めを解く
# text-to-text
t2t_generator = pipeline(
    "text2text-generation",
    model="retrieva-jp/t5-large-long",
)

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
# マスクされたスパンを予測
masked_text = "江戸幕府を開いたのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': '徳川家康'}]


In [7]:
t2t_generator.tokenizer.convert_tokens_to_ids("<extra_id_1>")

32098

In [8]:
masked_text = "日本で通貨を発行しているのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': '日本銀行'}]


In [9]:
"日本銀行" in t2t_generator.tokenizer.vocab

False

# 3.6 トークナイゼーション

## 3.6.1 バイト対符号化

In [10]:
# 単語と頻度
word_freqs = {
    "たのしい": 6,
    "たのしさ": 2,
    "うつくしい": 4,
    "うつくしさ": 1,
}
# 語彙を文字で初期化
vocab = sorted(set([char for word in word_freqs for char in word]))
# 単語ごとの分割状態
splits = {
    word: [char for char in word] for word in word_freqs
}

In [11]:
splits

{'たのしい': ['た', 'の', 'し', 'い'],
 'たのしさ': ['た', 'の', 'し', 'さ'],
 'うつくしい': ['う', 'つ', 'く', 'し', 'い'],
 'うつくしさ': ['う', 'つ', 'く', 'し', 'さ']}

In [13]:
from collections import Counter

def compute_most_frequent_pair(
    splits: dict[str, list[str]]
) -> tuple[str, str]:
    """最も頻度の高い隣接するサブワードの組を計算"""
    pair_freqs: Counter[tuple[str, str]] = Counter()
    for word, freq in word_freqs.items():
        split = splits[word]
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            # サブワードの組の頻度に単語の頻度を加算
            pair_freqs[pair] += freq
    # カウンタから最も頻度の高いサブワードの組を取得
    pair, _ = pair_freqs.most_common(1)[0]
    return pair

def merge_pair(
    target_pair: tuple[str, str],
    splits: dict[str, list[str]],
) -> dict[str, list[str]]:
    """サブワードの組を結合"""
    l_str, r_str = target_pair
    for word in word_freqs:
        split = splits[word]  # 現在の単語の分割状態
        i = 0
        while i < len(split) - 1:
            if split[i] == l_str and split[i + 1] == r_str:
                # i番目とi+1番目を結合して置き換える
                split = split[:i] + [l_str + r_str] + split[i + 2:]
            i += 1
        splits[word] = split
    return splits

In [14]:
for step in range(9):
    # 最も頻度の高い隣接するサブワードの組
    target_pair = compute_most_frequent_pair(splits)
    # 最も頻度の高い隣接するサブワードの組を結合
    splits = merge_pair(target_pair, splits)
    # 語彙にサブワードの組を追加
    vocab.append(target_pair)

In [15]:
print(vocab)

['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の', ('し', 'い'), ('た', 'の'), ('たの', 'しい'), ('う', 'つ'), ('うつ', 'く'), ('うつく', 'しい'), ('し', 'さ'), ('たの', 'しさ'), ('うつく', 'しさ')]
